In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../artifactory/')

In [2]:
import torch
import pickle
import warnings
import numpy as np
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.utils.data import DataLoader
from pathlib import Path
from itertools import repeat
from artifact import Saw
from data import ArtifactDataset, CachedArtifactDataset
from detector import ConvolutionDetector, WindowTransformerDetector
from utilities import parameters_k

# stop warnings
torch.set_float32_matmul_precision("high")
warnings.filterwarnings("ignore", ".*does not have many workers.*")


C:\Users\gustv\AppData\Roaming\Python\Python310\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

All settings.

In [4]:
# width of window
width = 512

# model
model = WindowTransformerDetector(window=width,                    
                                  convolution_features=[256, 128, 64, 32],
                                  convolution_width=[5, 9, 17, 33],
                                  convolution_dropout=0.0,
                                  transformer_heads=2,
                                  transformer_feedforward=128,
                                  transformer_layers=2,
                                  transformer_dropout=0,
                                  loss="mask",
                                  loss_boost_fp=0)
# model = ConvolutionDetector(convolution_features=[128, 64, 32],
#                             convolution_width=[5, 9, 33],
#                             convolution_dilation=[1, 1, 1],
#                             convolution_dropout=0.0,
#                             convolution_activation="sigmoid")
model_name = f"{model.__class__.__name__}_{parameters_k(model)}"

val_file = Path(f"../data/validation{width}.all.pkl")
val_datasets = [
    "australian_electricity_demand_dataset",
    "electricity_hourly_dataset",
    "electricity_load_diagrams",
    "HouseholdPowerConsumption1",
    "HouseholdPowerConsumption2",
    "london_smart_meters_dataset_without_missing_values",
    "solar_10_minutes_dataset",
    "wind_farms_minutely_dataset_without_missing_values",
]
train_datasets = [
    "australian_electricity_demand_dataset",
    "electricity_hourly_dataset",
    "electricity_load_diagrams",
    "HouseholdPowerConsumption1",
    "HouseholdPowerConsumption2",
    "london_smart_meters_dataset_without_missing_values",
    "solar_10_minutes_dataset",
    "wind_farms_minutely_dataset_without_missing_values",
]
print(model_name)

WindowTransformerDetector_528.96K


Loading data.

In [5]:
def load_series(names: list[str], split: str):
    series = list()
    counts = list()
    for name in names:
        with open(f"../data/processed/{name}_{split}.pickle", "rb") as f:
            raw = [a for a in pickle.load(f) if len(a) > width]
            series.extend(np.array(a).astype(np.float32) for a in raw)
            counts.extend(repeat(1 / len(raw), len(raw)))
    counts = np.array(counts)
    return series, counts / counts.sum()

In [6]:
# train
train_data, train_weights = load_series(train_datasets, "TRAIN")
train_dataset = ArtifactDataset(train_data,
                                Saw(min_width=4, max_width=32),
                                width=width,
                                padding=64,
                                weight=train_weights) 
train_loader = DataLoader(train_dataset, batch_size=32)

In [7]:
# validation
if not val_file.exists():
    val_data, val_weights = load_series(val_datasets, "TEST")
    val_gen = ArtifactDataset(val_data,
                              Saw(min_width=4, max_width=32),
                              width=width,
                              padding=64,
                              weight=val_weights)
    val = CachedArtifactDataset.generate(val_gen,
                                         n=2048,
                                         to=val_file)
else:
    val = CachedArtifactDataset(file=val_file)
val_loader = DataLoader(val, batch_size=32)

Sanity check.

In [8]:
batch = next(iter(train_loader))
batch["data"]

tensor([[0.0737, 0.0707, 0.0695,  ..., 0.4376, 0.4241, 0.3963],
        [0.1193, 0.1053, 0.1076,  ..., 0.1030, 0.1146, 0.0951],
        [0.0163, 0.0163, 0.0163,  ..., 0.2358, 0.2358, 0.2358],
        ...,
        [0.0382, 0.1649, 0.2842,  ..., 0.1776, 0.0992, 0.1025],
        [0.7148, 0.7454, 0.7397,  ..., 0.7006, 0.7183, 0.6949],
        [0.2881, 0.2478, 0.2284,  ..., 0.0000, 0.0000, 0.0000]])

In [9]:
model(batch["data"]).shape

torch.Size([32, 512])

Training!

In [10]:
# initialize callbacks
checkpointcallback = ModelCheckpoint(monitor="validation",
                                     mode="min",
                                     save_top_k=1)
lr_monitor = LearningRateMonitor(logging_interval='step')

# initialize logger
logger = TensorBoardLogger("models/detector",
                           name=model_name)

# initialize trainer
trainer = Trainer(logger=logger,
                  max_steps=50000,
                  val_check_interval=1000,
                  callbacks=[checkpointcallback,
                             lr_monitor])

# train
trainer.fit(model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: models/detector\WindowTransformerDetector_528.96K
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                        | Params
-------------------------------------------------------------
0 | convolutions | Sequential                  | 503 K 
1 | position     | SinusoidalPositionEmbedding | 0     
2 | dropout      | Dropout                     | 0     
3 | transformer  | TransformerEncoder          | 25.4 K
4 | linear       | Linear                      | 33    
-------------------------------------------------------------
528 K     Trainable params
0         Non-trainable params
528 K     Total params
2.116     Total estimated model params size (MB)


Epoch 0: : 20000it [15:31, 21.47it/s, v_num=0]                             Epoch 00020: reducing learning rate of group 0 to 1.0000e-04.
Epoch 0: : 41000it [32:26, 21.06it/s, v_num=0]Epoch 00041: reducing learning rate of group 0 to 1.0000e-05.
Epoch 0: : 47000it [37:48, 20.72it/s, v_num=0]Epoch 00047: reducing learning rate of group 0 to 1.0000e-06.
Epoch 0: : 50000it [40:20, 20.66it/s, v_num=0]

`Trainer.fit` stopped: `max_steps=50000` reached.


Epoch 0: : 50000it [40:20, 20.66it/s, v_num=0]
